In [2]:
##Helper T = Detector
##Killer T = Killer
##Unmarked Pathogen = Rabbits
##Marked Pathogen = Marked

import pycxsimulator
from pylab import *

width = 100
height = 100

initialRabbitPopulation = 5
rabbitReproductionRate = 0.05
rabbitPopulationLimit = 10**10
rabbitNoiseLevel = 3

initialMarkedPopulation = 0
markedReproductionRate = .1
markedPopulationLimit = 1000
markedNoiseLevel = 3

initialDetectorPopulation = 50
detectorReproductionLimit = 0.2 ##Not presently used
detectorPopulationLimit = [100,120]
detectorNoiseLevel = 2.5
detectorHungerLimit = 10

initialFoxPopulation = 20
foxReproductionLimit = 20
foxPopulationLimit = 100
foxNoiseLevel = 2.5
foxHungerLimit = 10

collisionDistance = 2
CDsquared = collisionDistance ** 2
toBeRemoved = -1

def initialize():
    global time, rabbits, foxes, rabbitData, foxData, detectors, detectorData, marked, markedData

    time = 0
    
    rabbits = []
    for i in range(initialRabbitPopulation):
        rabbits.append([uniform(0, width), uniform(0, height)])

    foxes = []
    for i in range(initialFoxPopulation):
        foxes.append([uniform(0, width), uniform(0, height), 0])
        
    detectors = []
    for i in range(initialDetectorPopulation):
        detectors.append([uniform(0, width), uniform(0, height),round(10*random())])   
        
    marked = []
    for i in range(initialMarkedPopulation): #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        Marked.append([uniform(0, width), uniform(0, height)]) #Can Probably be Deleted, Only Useful if starting simulation mid-infection

    rabbitData = [initialRabbitPopulation]
    foxData = [initialFoxPopulation]
    detectorData = [initialDetectorPopulation]
    markedData = [initialMarkedPopulation]

def observe():
    subplot(1, 2, 1)
    cla()
    if rabbits != []:
        x = [ag[0] for ag in rabbits]
        y = [ag[1] for ag in rabbits]
        scatter(x, y, color = 'red')
    if marked != []:
        x = [ag[0] for ag in marked]
        y = [ag[1] for ag in marked]
        scatter(x, y, color = 'pink')
    if foxes != []:
        x = [ag[0] for ag in foxes]
        y = [ag[1] for ag in foxes]
        scatter(x, y, color = 'green')
    if detectors != []:
        x = [ag[0] for ag in detectors]
        y = [ag[1] for ag in detectors]
        scatter(x, y, color = 'blue')
    axis('scaled')
    axis([0, width, 0, height])
    title('t = ' + str(time))

    subplot(1, 2, 2)
    cla()
    plot(rabbitData, color = 'red')
    plot(markedData, color = 'pink')
    plot(foxData, color = 'green')
    plot(detectorData, color = 'blue')
    title('Populations')

def clip(a, amin, amax):
    if a < amin: return amin
    elif a > amax: return amax
    else: return a

def update():
    global time, rabbits, foxes, rabbitData, foxData, detectors, detectorData, marked, markedData

    time += 1
    
    # simulate random motion
    
    for i in range(len(rabbits)):
        rabbits[i][0] = rabbits[i][0] + normal(0, rabbitNoiseLevel)
        rabbits[i][1] = rabbits[i][1] + normal(0, rabbitNoiseLevel)
        rabbits[i][0] = clip(rabbits[i][0], 0, width)
        rabbits[i][1] = clip(rabbits[i][1], 0, height)

    #for ag in rabbits:
     #   ag[0] += normal(0, rabbitNoiseLevel)
     #   ag[1] += normal(0, rabbitNoiseLevel)
     #   ag[0] = clip(ag[0], 0, width)
     #   ag[1] = clip(ag[1], 0, height)
     #   print(ag)
    #print('---')
    
    for ag in marked:
        ag[0] += normal(0, markedNoiseLevel)
        ag[1] += normal(0, markedNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in foxes:
        ag[0] += normal(0, foxNoiseLevel)
        ag[1] += normal(0, foxNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in detectors:
        ag[0] += normal(0, detectorNoiseLevel)
        ag[1] += normal(0, detectorNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
    
    # detect collision and change state
    killing_locations = []
    for i in range(len(foxes)):
        foxes[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (foxes[i][0]-marked[j][0])**2 + (foxes[i][1]-marked[j][1])**2 < CDsquared:
                    killing_locations.append([(foxes[i][0]+marked[j][0])/2,(foxes[i][1]+marked[j][1])/2])
                    marked[j] = toBeRemoved          # Pathogen is Killed
        if foxes[i][2] > foxHungerLimit:
            foxes[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
    
    marking_locations = []
    for i in range(len(detectors)):
        detectors[i][2] += 1                      # Detector Cell's Age increasing
        for j in range(len(rabbits)):
            if rabbits[j] != toBeRemoved:
                if (detectors[i][0]-rabbits[j][0])**2 + (detectors[i][1]-rabbits[j][1])**2 < CDsquared:
                    marking_locations.append([(detectors[i][0]+rabbits[j][0])/2,(detectors[i][1]+rabbits[j][1])/2])
                    marked.append(rabbits[j]) #Marked Pathogen entity is created
                    rabbits[j] = toBeRemoved # Unmarked Pathogen entity is removed
        if detectors[i][2] > detectorHungerLimit:
            detectors[i] = toBeRemoved            # Detector Cell Lifespan Exceeded
        
    # remove "toBeRemoved" agents
    while toBeRemoved in rabbits:
        rabbits.remove(toBeRemoved)
    while toBeRemoved in marked:
        marked.remove(toBeRemoved)
    while toBeRemoved in foxes:
        foxes.remove(toBeRemoved)
    while toBeRemoved in detectors:
        detectors.remove(toBeRemoved)

    # count survivors' populations
    rabbitPopulation = len(rabbits)
    markedPopulation = len(marked)
    foxPopulation = len(foxes)
    detectorPopulation = len(detectors)

    # produce offspring
    for i in range(len(rabbits)):
        if random() < rabbitReproductionRate:
            offspring = rabbits[i].copy()
            rabbits.append(offspring) # making and adding a copy of the parent
            rabbitPopulation +=1
    for i in range(len(marked)):
        if random() < markedReproductionRate:
            offspring = marked[i].copy()
            rabbits.append(offspring) # making and adding a copy of the parent, new cell is unmarked
            rabbitPopulation +=1
    
    ##This loop is important but something about it keeps crashing python
    
    #newFoxesToday = 0
    #while len(foxes)<initialFoxPopulation or newFoxesToday < min(foxReproductionLimit,len(marking_locations)):
    #    if len(killing_locations)>1:
    #        random_choice = killing_locations[int(len(killing_locations)*random()//1)]
    #        angle = 2*pi*rand()
    #        New_X = random_choice[0]+5*rand()*sin(angle)
    #        New_Y = random_choice[1]+5*rand()*cos(angle)
    #    else:
    #        New_X = 100*random()
    #        New_Y = 100*random()
    #    offspring = [New_X,New_Y,0]
    #    offspring[0] = clip(offspring[0], 0, width)
    #    offspring[1] = clip(offspring[1], 0, height)
    #   foxes.append(offspring) # making and adding a copy of the parent
    
    for i in range(20):
        if len(marking_locations)>1:
            random_choice = marking_locations[int(len(marking_locations)*random()//1)]
            New_X = random_choice[0]+normal(0,5)
            New_Y = random_choice[1]+normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        if len(detectors)<detectorPopulationLimit[0] or random() < .5 and len(detectors)<detectorPopulationLimit[1] :
            offspring = [New_X,New_Y,0]
            offspring[0] = clip(offspring[0], 0, width)
            offspring[1] = clip(offspring[1], 0, height)
            detectors.append(offspring) # making and adding a copy of the parent

    rabbitData.append(len(rabbits))
    foxData.append(len(foxes))
    detectorData.append(len(detectors))
    markedData.append(len(marked))

pycxsimulator.GUI().start(func=[initialize, observe, update])

In [5]:
cos(3*pi)

NameError: name 'cos' is not defined

In [4]:
max(2,3)

3